# Direction Finding

The direction of the sound source is here estimated by determining for the same signal the relative delays between the different hydrophones and by considering the following relation between measured delays and the direction vector of the sound source. 

Given a vector of arrival delays $\Delta \bm{\tau} $  one estimates the sound direction  $\bm{\hat s}$   by

\begin{equation}
    \bm{\hat s} = c\left(\bm D_0^T \bm D_0 \right)^{-1} \bm D_0 \Delta \bm \tau
\end{equation}

where $c$ is the sound speed between the hydrophones,

\begin{equation}
    \bm{\hat s} = 
    \left(
        \begin{array}{c}
            s_x\\
            s_y\\
            s_z
        \end{array}
    \right)  
\end{equation}



\begin{equation}
    \Delta \bm \tau = 
    \left(
        \begin{array}{c}
            \Delta \tau_1\\
            \Delta \tau_2\\
            \vdots\\
            \Delta \tau_n
        \end{array}
    \right)  
    \text{ is the vector of arrival delays, and}
\end{equation}



\begin{equation}
    \bm D_0 = 
    \left(
        \begin{array}{c c c c}
            d_{1x} & d_{2x} & ... & d_{nx}\\
            d_{1y} & d_{2y} & ... & d_{ny}\\
            d_{1z} & d_{2z} & ... & d_{nz}\\
        \end{array}
    \right)  
\end{equation}

is the hydrophone separation matrix.


The arrival angles azimuth $\alpha$ and elevation $\beta$  of the sound are finally estimated by solving

\begin{equation}
    \tan \alpha = \frac{-s_y}{s_x}
\end{equation}

and

\begin{equation}
    \sin \beta = s_z
\end{equation}

or alternatively

\begin{equation}
    \tan \beta = \frac{s_z}{\sqrt{s_x^2+s_y^2}}
\end{equation}

As the azimuth angles may vary from -180°  to 180°  the solutions require the usual care for  $s_x<0$, i.e. by using the four-quadrant inverse tangent. 
The minus sign in equation is simply due to the convention that positive azimuth is measured towards starboard. 

As the elevation angle $\beta$ is restricted to ±90°, an ordinary inverse sinus is sufficient but a four-quadrant inverse tangent seems also here preferred because it is insensitive to a common factor (e.g. uncertain sound speed at the array).

The minimal number of hydrophones to form a volumetric array is 4 allowing to form n = 6 different hydrophone pairs.
To obtain an optimal volumetric array the hydrophones are located on the tips of an tetrahedron. This way all 6 hydrophone pairs have the same sepatation.
While direction finding could be done with only 3 hydrophone pairs, the use of all 6 different combinations should improve the performance, as the impact of measurement errors is reduced. 
This is reflected by the Least-Mean-Square notation in equation. 
Also, using all possible hydrophone pairs puts the reference location of the array automatically in the center of the array eliminating the need for a selection of a reference hydrophone.

A volumetric array with 6 hydrophones allows the formation of n = 15 different pairs of hydrophones. 

